In [ ]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# partly from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Loading Data

In [ ]:
with open("options.txt", 'r') as f:
    options = f.readlines()
    options = {option.split("=")[0]: option.split("=")[1].strip() for option in options}
print(options)

In [ ]:
RUG = pd.read_pickle(options['RUG_no_outliers'])
RUG.interpolate(method='linear', inplace=True)
RUG = RUG[::10]
dfs = [RUG.filter([i]) for i in RUG]

In [ ]:
# create list of seperated dataframe columns and drop NaN values
dfs = [i.dropna() for i in dfs]

# Timeseries LSTM

In [ ]:
def func(df, name):
    # copy data to avoid changing original data
    df2 = df.copy()

    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(df2)

    # split into train and test sets
    train_size = int(len(dataset) * 0.8)
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


    # convert a datapoint (time series) to vertical format and 
    # add column for previous known values using look_back
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    # trainsform training and test datasets into forms usable by LSTM
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    # create model callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001, verbose=2)

    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.fit(trainX, trainY, epochs=50, verbose=2, callbacks=[early_stopping, reduce_lr])

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # invert scale of data back to original range
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])

    # calculate metrics
    rmse_train = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    rmse_test = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))

    mae_train = tf.keras.metrics.mean_absolute_error(trainY[0], trainPredict[:,0]).numpy()
    mae_test = tf.keras.metrics.mean_absolute_error(testY[0], testPredict[:,0]).numpy()

    mape_train = tf.keras.metrics.mean_absolute_percentage_error(trainY[0], trainPredict[:,0]).numpy()
    mape_test = tf.keras.metrics.mean_absolute_percentage_error(testY[0], testPredict[:,0]).numpy()

    return (name, (rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test))

In [ ]:
results = []

# amount of iterations to use for the average
n_iterations = 3

In [ ]:
for _ in range(n_iterations):
    # iterate over all columns and run function
    for name, df in zip([i for i in RUG], dfs):
        print(name)
        
        # incase of error with removing NaN values, skip
        if df.isnull().values.any():
            print("NaN")
            results.append((name, (np.nan, np.nan)))
            print("-------------")
            continue
        
        # run function for baseline algorithm
        r = func(df, name)
        print("-------------")
        results.append(r)

In [ ]:
# make dataframe from results
lstm_results = [[i[0], i[1][0], i[1][1], i[1][2], i[1][3], i[1][4], i[1][5]] for i in results]
lstm_results = pd.DataFrame(lstm_results)

# add column names and set index
lstm_results.columns = ["Location", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train MAPE", "Test MAPE"]
lstm_results.set_index("Location", inplace=True)
lstm_results.sort_index(inplace=True)

# get average scores for each location
lstm_results = lstm_results.astype(float)
lstm_results = lstm_results.groupby('Location').mean()

# create multiindex column names
cols = pd.MultiIndex.from_product([['Train', 'Test'], ['RMSE', 'MAE', 'MAPE']])
lstm_results.columns = cols

display(lstm_results)

print(lstm_results.round(3).to_latex())